In [1]:
import datetime
import sqlalchemy
import sqlalchemy.orm
import pandas as pd

Параметры подключения (сервер, БД, драйвер)

In [2]:
SERVER = 'KONSTANTIN\SQLEXPRESS'
DB = 'testdb'
DRIVER = 'SQL Server'

In [3]:
conn = sqlalchemy.create_engine(
    f'mssql+pyodbc://@{SERVER}/{DB}?trusted_connection=yes&driver={DRIVER}')
session = sqlalchemy.orm.Session(conn)

**Создание процедуры (питон)**

In [4]:
# чтобы не засорять код, текст запроса грузим из файла
with session.begin():
    conn.execute(open('.\SQL_Задание_3.sql', encoding='utf_8_sig').read())

**Запросы**

In [5]:
def fill_contractor_table(dt_from: str, dt_to: str) -> None:
    """Заполняет таблицу SHIFT_CALC данными о сменах за период
    
    Args:
      dt_drom - дата начала
      dt_to - дата окончания (включительно)
    """
    cursor = conn.raw_connection().cursor()
    cursor.execute('SHIFT_CALC ?, ?', [dt_from, dt_to])
    cursor.commit()
    
def read_contractor_table() -> pd.DataFrame:
    """Загружает данные таблицы SHIFT_CALC в Pandas
    
    Returns:
      pandas dataframe
    """
    data = pd.read_sql_table('T_CONTRACTOR_WORK_DAY',
                             con=conn,
                             index_col='ID',
                             parse_dates=['DATE_BEGIN', 'DATE_END'])
    return data

Поскольку в задании не указано, какие методы обработки следует использовать, я решил загрузить таблицу в pandas и обработать данные его методами. Чистый SQL уже реализован в скриптах

**Запрос 1**

В задании не указан период, для примера беру февраль 2019

In [6]:
fill_contractor_table('2019-02-01', '2019-02-28')
query_1 = read_contractor_table()
result_1 = query_1.groupby('NAME')['DATE_BEGIN'].count()
result_1

NAME
Поставщик 2    20
Поставщик 3    14
Name: DATE_BEGIN, dtype: int64

**Запрос 2**

In [7]:
fill_contractor_table('2019-01-01', '2019-01-31')
query_2 = read_contractor_table()
gb = query_2.groupby('NAME')['DATE_BEGIN'].count()
result_2 = gb[gb > 10]
result_2

NAME
Поставщик 1    15
Поставщик 2    22
Поставщик 3    16
Name: DATE_BEGIN, dtype: int64

**Запрос 3**

<div class="alert alert-warning">
    
Важное замечание

Здесь я не учитываю смены, которые начались в предыдущие сутки и закончились
14, 15 или 16 января. В SQL запросе, наоборот, их учитывал
    
</div>

In [8]:
fill_contractor_table('2019-01-13', '2019-01-16')
query_3 = read_contractor_table()
query_3['DATE_BEGIN'] = query_3['DATE_BEGIN'].dt.date
query_3['DATE_END'] = query_3['DATE_END'].dt.date

dt1 = set(query_3.loc[query_3['DATE_BEGIN'] == datetime.date.fromisoformat('2019-01-14'), 'NAME'])
dt2 = set(query_3.loc[query_3['DATE_BEGIN'] == datetime.date.fromisoformat('2019-01-15'), 'NAME'])
dt3 = set(query_3.loc[query_3['DATE_BEGIN'] == datetime.date.fromisoformat('2019-01-16'), 'NAME'])
result = dt1.intersection(dt2).intersection(dt3)
result

{'Поставщик 1', 'Поставщик 2'}